<a href="https://colab.research.google.com/github/Baah134/Baah134/blob/main/Whisper/Whisper_Transcription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [2]:
import datasets

print(datasets.__version__)

3.2.0


In [ ]:
from datasets import load_dataset

afrispeech = load_dataset("tobiolatunji/afrispeech-200", 'all', split="train", streaming=True)

print(next(iter(afrispeech)))
print(list(afrispeech.take(100)))


In [4]:
for sample in afrispeech.take(1):  # Use .take(1) to get the first sample
    audio = sample["audio"]  # Access the audio data
    transcription = sample["transcript"]  # Access the transcription

    # Display information
    print("Audio Path:", audio["path"])  # Path to the audio file (if available)
    print("Audio Data:", audio["array"][:10], "...")  # Preview of audio waveform
    print("Sampling Rate:", audio["sampling_rate"])  # Sampling rate of the audio
    print("Transcription:", transcription)


Reading metadata...: 57819it [00:01, 43453.89it/s]


Audio Path: e54c96e9-6085-46a8-82c3-0c7efe10360d/bcc0988c06ec27f8622bc259c8d67a3d.wav
Audio Data: [-1.95312500e-03 -1.73950195e-03 -1.55639648e-03 -1.31225586e-03
 -1.12915039e-03 -9.15527344e-04 -7.32421875e-04 -5.18798828e-04
 -3.05175781e-04 -9.15527344e-05] ...
Sampling Rate: 44100
Transcription: Similarly, an ACEI or a sartan is preferred over other antihypertensive drugs in diabetic patients where they slow the progression of nephropathy.


In [ ]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg


In [ ]:
!pip install jiwer

In [7]:
from jiwer import wer
import whisper
import numpy as np

model = whisper.load_model("medium")
total_samples = 5
wer_scores = []

for sample in afrispeech.take(total_samples):
    # Ground truth
    ground_truth = sample['transcript']

    # Audio waveform (convert to float32)
    audio_array = np.array(sample['audio']['array'], dtype=np.float32)

    # Transcribe the audio
    result = model.transcribe(audio_array, fp16=False)
    predicted_text = result['text']

    # Calculate WER for the sample
    error_rate = wer(ground_truth, predicted_text)
    wer_scores.append(error_rate)

    # Print results for the sample
    print(f"Ground Truth: {ground_truth}")
    print(f"Predicted Text: {predicted_text}")
    print(f"WER: {error_rate}\n")

# Calculate average WER across samples
average_wer = sum(wer_scores) / len(wer_scores)
print(f"Average WER across {total_samples} samples: {average_wer}")

100%|█████████████████████████████████████| 1.42G/1.42G [00:19<00:00, 77.3MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Ground Truth: Similarly, an ACEI or a sartan is preferred over other antihypertensive drugs in diabetic patients where they slow the progression of nephropathy.
Predicted Text: 
WER: 1.0

Ground Truth: The moisture penetrates the sterile cloth, or paper, and carries organisms by capillary action to contaminate the eld.
Predicted Text: 
WER: 1.0

Ground Truth: Various sites may be used for subcutaneous injections, including the outer aspect of the upper arm, the abdomen from below the costal margin to the iliac crests, the anterior aspects of the thigh, the upper back and the upper ventral gluteal area.
Predicted Text: 
WER: 1.0

Ground Truth: The major causes are as follows:iSystemic hypertensioni i Mitral or aortic valve disease stenosisiii Ischaemic heart diseaseiv Myocardial diseases e.

Predicted Text:  Erase Her reaches or cheeks Her Lungs the quicksand twitching may becomeaja purposes of healing
WER: 1.0

Average WER across 5 samples: 1.0
